In [1]:
# boiler plate
from collections import Counter
import pickle
import os
from importlib import reload
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler


import ipdb
import mytf.utils as mu

In [2]:
with open('models/2019-07-21T1815-UTC-outdata-SUBSET50k.pkl', 'rb') as fd:
    julydata = pickle.load(fd)


In [3]:
class_weights = {0: 1.0, 1: 1.3, 2: 1.0, 3: 1.0}
training_indices = np.arange(0, julydata['x_train'].shape[0], 1)
print('training indices...', training_indices.shape)

dataset_batches = mu.build_dataset_weighty_v3(
    julydata,
    training_indices, 
    class_weights,
    batch_size=1000)

training indices... (50000,)
Start build v3: .. doesnt add up to 1.0
num slices 50
size_remainder,  0
Counter({0: 521, 2: 404, 3: 62, 1: 13})
weights_per_class,  Tensor("truediv:0", shape=(4,), dtype=float32)
1003.8999999999996
Counter({0: 532, 2: 382, 3: 64, 1: 22})
weights_per_class,  Tensor("truediv_1:0", shape=(4,), dtype=float32)
1006.5999999999997
Counter({0: 513, 2: 411, 3: 57, 1: 19})
weights_per_class,  Tensor("truediv_2:0", shape=(4,), dtype=float32)
1005.6999999999996
Counter({0: 557, 2: 386, 3: 35, 1: 22})
weights_per_class,  Tensor("truediv_3:0", shape=(4,), dtype=float32)
1006.5999999999993
Counter({0: 521, 2: 411, 3: 48, 1: 20})
weights_per_class,  Tensor("truediv_4:0", shape=(4,), dtype=float32)
1005.9999999999995
Counter({0: 519, 2: 408, 3: 54, 1: 19})
weights_per_class,  Tensor("truediv_5:0", shape=(4,), dtype=float32)
1005.6999999999995
Counter({0: 543, 2: 394, 3: 46, 1: 17})
weights_per_class,  Tensor("truediv_6:0", shape=(4,), dtype=float32)
1005.0999999999997
Coun

In [10]:
def full_model():
    with tf.name_scope('themodel'):
        model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64,   dropout=0.2, recurrent_dropout=0.2,
                    batch_input_shape=(None, 256, 3), 
                  ),
        # 4 because 'A', 'B', 'C', 'D'.
        tf.keras.layers.Dense(4)])
        return model


In [7]:
julydata['y_train'][:5, :]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [24]:

training_indices = np.arange(0, julydata['x_train'].shape[0], 1)
print('training indices...', training_indices.shape)

train_vec, label_vec, weights_vec = mu.helper_build_dataset_weighty_v3(
    julydata,
    training_indices, 
    class_weights,
    batch_size=1000)

training indices... (50000,)
Start build v3: .. doesnt add up to 1.0
num slices 50
size_remainder,  0
Counter({0: 524, 2: 398, 3: 59, 1: 19})
weights_per_class,  Tensor("truediv:0", shape=(4,), dtype=float32)
1005.6999999999998
Counter({0: 547, 2: 385, 3: 52, 1: 16})
weights_per_class,  Tensor("truediv_1:0", shape=(4,), dtype=float32)
1004.7999999999995
Counter({0: 554, 2: 384, 3: 47, 1: 15})
weights_per_class,  Tensor("truediv_2:0", shape=(4,), dtype=float32)
1004.4999999999998
Counter({0: 507, 2: 427, 3: 54, 1: 12})
weights_per_class,  Tensor("truediv_3:0", shape=(4,), dtype=float32)
1003.5999999999999
Counter({0: 530, 2: 411, 3: 46, 1: 13})
weights_per_class,  Tensor("truediv_4:0", shape=(4,), dtype=float32)
1003.8999999999996
Counter({0: 511, 2: 415, 3: 48, 1: 26})
weights_per_class,  Tensor("truediv_5:0", shape=(4,), dtype=float32)
1007.7999999999994
Counter({0: 540, 2: 397, 3: 47, 1: 16})
weights_per_class,  Tensor("truediv_6:0", shape=(4,), dtype=float32)
1004.7999999999997
Coun

In [54]:
LOGDIR = './runtime/'

def mnist_model(learning_rate, blahname):
    tf.reset_default_graph()
    sess = tf.Session()
    run_metadata = tf.RunMetadata()
        
    # Setup placeholders, and reshape the data
    x = tf.placeholder(tf.float32, shape=[None, 256, 3], name="x")
        
    # x_image = tf.reshape(x, [-1, 28, 28, 1])
    # ? tf.summary.image('input', x_image, 3)
    y = tf.placeholder(tf.int64, shape=[None, 1], name="labels")

    themodel = full_model()
    logits = themodel(x) # training=True ?
    
    tf.summary.histogram("logits_0", logits[:, 0]) # NOTE might need to use mean?
    tf.summary.histogram("logits_1", logits[:, 1])
    tf.summary.histogram("logits_2", logits[:, 2])
    tf.summary.histogram("logits_3", logits[:, 3])


    with tf.name_scope("xent"):
        xent = tf.losses.sparse_softmax_cross_entropy(y, logits, 
                                                      #weights=weights
                                                     )
        # xent = tf.reduce_mean(
        #         tf.nn.softmax_cross_entropy_with_logits(
        #                 logits=logits, labels=y), name="xent")
        tf.summary.scalar("xent", xent)
        tf.summary.histogram("x_entropy", xent)
        

    with tf.name_scope("train"):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), y)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()


    # embedding = tf.Variable(tf.zeros([1024, embedding_size]), name="test_embedding")
    # assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(LOGDIR + '/' + blahname) # <--hyperparam too.
    writer.add_graph(sess.graph)
    
    writer.add_run_metadata(run_metadata, 'blahhmmm')

    # config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    # embedding_config = config.embeddings.add()
    # embedding_config.tensor_name = embedding.name
    # embedding_config.sprite.image_path = SPRITES
    # embedding_config.metadata_path = LABELS
    # Specify the width and height of a single thumbnail.
    # embedding_config.sprite.single_image_dim.extend([28, 28])
    # tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)

    # for (i, batch) in enumerate(dataset_batches):

    for i in range(50): # 45 or 50?
        #batch = dataset_batches.next_batch(1000)
        left, right = train_vec[i], np.resize(label_vec[i], (1000,1))  # , weights_vec

        
        if i % 5 == 0:
            [train_accuracy, s] = sess.run([accuracy, summ], 
                                           feed_dict={x: left,
                                                      y: right})
            writer.add_summary(s, i)
        if i % 5 == 0:
            #sess.run(assignment, feed_dict={x: mnist.test.images[:1024], 
            #                                y: mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(LOGDIR, blahname, "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: left, y: right}, run_metadata=run_metadata)

        
def main(blahname):
    learning_rate = 1E-3
    mnist_model(learning_rate, blahname)
    print('Done training!')
    print('Run `tensorboard --logdir=%s` to see the results.' % LOGDIR)
    print('Running on mac? If you want to get rid of the dialogue asking to give '
                'network permissions to TensorBoard, you can provide this flag: '
                '--host=localhost')




Getting error when running my `main()`
```python
ValueError("Cannot feed value of shape (1000,) for Tensor 'labels:0', which has shape '(?, 1)'")```

I think problem is somewhere along these lines...
```python
ipdb> p type(right)
<class 'numpy.ndarray'>
ipdb> p right.shape
(1000,)
ipdb> p right[:3]
array([1, 2, 0])
ipdb> p np.resize(right, (1000,1)).shape
(1000, 1)
ipdb> p np.resize(right, (1000,1))[:5]
array([[1],
       [2],
       [0],
       [2],
       [0]])
```

In [52]:
import datetime; import pytz
def quickts():
    return datetime.datetime.utcnow().replace(tzinfo=pytz.UTC).strftime('%Y-%m-%dT%H%M%S')
quickts()

'2019-08-11T174832'

In [56]:
%%time
main(quickts())

Instructions for updating:
Use standard file APIs to delete files with this prefix.
Done training!
Run `tensorboard --logdir=./runtime/` to see the results.
Running on mac? If you want to get rid of the dialogue asking to give network permissions to TensorBoard, you can provide this flag: --host=localhost
CPU times: user 3min 51s, sys: 1min 35s, total: 5min 27s
Wall time: 1min 15s


### Outputs from tensorboard
* Wow that Tensorboard histogram output is so cool... 
* so this number `40` is showing up because of the batch number `i` in the code . It should have gone up to `45` I think, but anyhow.
* So here just like in my matplot lib notes from [2019-07-21](https://github.com/namoopsoo/aviation-pilot-physiology-hmm/blob/master/notes/2019-07-21.ipynb), it's looking like class=1 and class=3 are outputting negative values, but the nice part about this tensorboard output is that it's actually showing the progression over the batches. And of course it makes sense that at the beginning we start from histograms for all logit outputs, around `0`.

<img src="assets/Screen%20Shot%202019-08-11%20at%202.27.23%20PM.png" width="347" height="262">

<img src="assets/Screen%20Shot%202019-08-11%20at%202.36.13%20PM.png" width="40%" height="40%">

<img src="assets/Screen%20Shot%202019-08-11%20at%202.36.28%20PM.png" width="40%" height="40%">

<img src="assets/Screen%20Shot%202019-08-11%20at%202.36.38%20PM.png" width="40%" height="40%">


